# Data Analytical Report
Reporting the result of the analysis

In [71]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

roundTrip_profit_g = pd.read_csv('../data/temproary_data/roundTrip_profit.csv')
roundTrip_profit_g['round_trip_route_IATA'] = roundTrip_profit_g['round_trip_route_IATA'].apply(lambda x: eval(x))
roundTrip_profit_g['inbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['inbound_FL_DATE'])
roundTrip_profit_g['outbound_FL_DATE'] = pd.to_datetime(roundTrip_profit_g['outbound_FL_DATE'])

roundTrip_fre_g = pd.read_csv('../data/temproary_data/round_trip_flights.csv')
roundTrip_fre_g['round_trip_route_IATA'] = roundTrip_fre_g['round_trip_route_IATA'].apply(lambda x: eval(x))

## Evaluate the stability of the plane ticket itself

- weather the revenue is stable
- weather the cost is stable
- weather the profit is stable

In [72]:
# scaler = StandardScaler()


# def cal_weekly_stats(df) -> pd.DataFrame:
#     if df.shape[0] < 14:
#         return [np.NaN, np.NaN, np.NaN]

#     # bin data by date
#     df = df.set_index('pruchse_date')
#     df = df.resample('1D').agg({'total_cost':'sum', 'total_revenue':'sum', 'profit':'sum'})

#     # calculate moving average of 7 days window
#     df = df.rolling(window=7).mean().reset_index()

#     # calculate the standard deviation of the moving average of cost, revenue and profit
#     return [df['total_cost'].std(), df['total_revenue'].std(), df['profit'].std()]



# new_roundTrip_profit = roundTrip_profit.assign(pruchse_date = np.minimum(roundTrip_profit['inbound_FL_DATE'], roundTrip_profit['outbound_FL_DATE']))
# new_roundTrip_profit['total_cost'] = scaler.fit_transform(roundTrip_profit[['total_cost']])
# new_roundTrip_profit['total_revenue'] = scaler.fit_transform(roundTrip_profit[['total_revenue']])
# new_roundTrip_profit['profit'] = scaler.fit_transform(roundTrip_profit[['profit']])

# profit_stability = new_roundTrip_profit.groupby('round_trip_route_IATA')[['pruchse_date','total_cost', 'total_revenue', 'profit']].apply(cal_weekly_stats).reset_index()

# profit_stability = profit_stability.assign(rolling_avg_cost = profit_stability[0].apply(lambda x: x[0]))
# profit_stability = profit_stability.assign(rolling_avg_cost_rank = profit_stability['rolling_avg_cost'].rank(ascending=True))
# profit_stability = profit_stability.assign(rolling_avg_revenue = profit_stability[0].apply(lambda x: x[1]))
# profit_stability = profit_stability.assign(rolling_avg_revenue_rank = profit_stability['rolling_avg_revenue'].rank(ascending=True))
# profit_stability = profit_stability.assign(rolling_avg_profit = profit_stability[0].apply(lambda x: x[2]))
# profit_stability = profit_stability.assign(rolling_avg_profit_rank = profit_stability['rolling_avg_profit'].rank(ascending=True))
# profit_stability.drop(columns=0, inplace=True)

In [73]:
new_roundTrip_profit = roundTrip_profit_g.assign(purchase_date = np.minimum(roundTrip_profit_g['inbound_FL_DATE'], roundTrip_profit_g['outbound_FL_DATE']))
new_roundTrip_profit = new_roundTrip_profit.groupby(['round_trip_route_IATA', pd.Grouper(key = 'purchase_date', freq='100D')])['profit'].mean()

new_roundTrip_profit =  new_roundTrip_profit.reset_index().groupby('round_trip_route_IATA').mean().rename(columns={'profit':'weekly_avg_profit'})
new_roundTrip_profit['weekly_avg_rank'] = new_roundTrip_profit['weekly_avg_profit'].rank(ascending=False, method='min')
# new_roundTrip_profit = new_roundTrip_profit[['weekly_avg_rank', 'profit']]
new_roundTrip_profit = new_roundTrip_profit.reset_index()

In [74]:
new_roundTrip_profit

,round_trip_route_IATA,weekly_avg_profit,weekly_avg_rank
0,"(ABE, ATL)",44051.677068,158.0
1,"(ABE, CLT)",39060.526414,276.0
2,"(ABE, DTW)",13615.185832,1418.0
3,"(ABE, FLL)",-4426.697750,2488.0
4,"(ABE, ORD)",35733.344067,364.0
...,...,...,...
2770,"(STL, TPA)",7939.627630,1834.0
2771,"(STL, TUL)",15767.382440,1286.0
2772,"(SYR, TPA)",-4749.308920,2500.0
2773,"(TPA, TTN)",-22160.059612,2761.0


## Aggregrating all the factors and assign ranks to each factors for round-trip route
consider factors:
- flight frequency
- flight profit (total profit - total cost)
- TODO: weekly profit

In [75]:
# calculate the ranking of the profit
roundTrip_profit = (roundTrip_profit_g.groupby('round_trip_route_IATA')['profit']
                 .sum()
                 .reset_index()
                 .rename(columns={'profit':'round_trip_profit'}))
roundTrip_profit = (roundTrip_profit.assign(round_trip_profit_ranking = roundTrip_profit['round_trip_profit']
                                    .rank(ascending = False, method = 'min')))


# calculate the ranking of the frequency
roundTrip_fre = (roundTrip_fre_g.groupby('round_trip_route_IATA')['outbound_OCCUPANCY_RATE']
                 .count().sort_values(ascending=False)
                 .reset_index()
                 .rename(columns={'outbound_OCCUPANCY_RATE':'round_trip_flights_count'}))
roundTrip_fre = (roundTrip_fre.assign(round_trip_flights_count_ranking = roundTrip_fre['round_trip_flights_count']
                              .rank(ascending = False, method = 'min')))


# calculate the ranking of the operation career
roundTrip_op = (roundTrip_profit_g
                .groupby('round_trip_route_IATA')
                .apply(lambda df: len(set(df['inbound_OP_CARRIER']).union(set(df['outbound_OP_CARRIER']))))
                .reset_index()
                .rename(columns={0:'round_trip_op_count'}))
# roundTrip_op = (roundTrip_op.assign(round_trip_op_count_ranking = roundTrip_op['round_trip_op_count']
#                               .rank(ascending = True, method = 'min')))


# # merge the ranks
round_trip_info = roundTrip_profit.merge(roundTrip_fre, on='round_trip_route_IATA', how='inner')

round_trip_info = round_trip_info.merge(new_roundTrip_profit, on='round_trip_route_IATA', how='left')

round_trip_info = round_trip_info.merge(roundTrip_op, on='round_trip_route_IATA', how='left')

# # # aggregate the ranking
round_trip_info = round_trip_info.assign(avg_ranking = ((round_trip_info['round_trip_flights_count_ranking'] * 0.15 + round_trip_info['round_trip_profit_ranking'] * 0.8) + round_trip_info['weekly_avg_rank'] * 0.05)/3)
round_trip_info.sort_values(by='avg_ranking', ascending=True).head(20)


,round_trip_route_IATA,round_trip_profit,round_trip_profit_ranking,round_trip_flights_count,round_trip_flights_count_ranking,weekly_avg_profit,weekly_avg_rank,round_trip_op_count,avg_ranking
1555,"(EWR, SFO)",5.832132e+07,3.0,1202,94.0,48520.229339,100.0,2,7.166667
1978,"(JFK, LAX)",1.065609e+08,1.0,3140,4.0,33936.599985,422.0,4,7.500000
128,"(ATL, CLT)",4.717478e+07,7.0,1534,43.0,30752.787865,535.0,3,12.933333
831,"(CLT, GSP)",5.748892e+07,4.0,772,255.0,74467.518566,8.0,1,13.950000
1382,"(DSM, ORD)",3.914057e+07,13.0,979,148.0,39980.158028,251.0,8,15.050000
830,"(CLT, GSO)",5.242073e+07,5.0,743,281.0,70552.795639,12.0,3,15.583333
733,"(CHS, CLT)",4.335832e+07,10.0,784,243.0,55303.975038,48.0,2,15.616667
1109,"(DCA, ORD)",4.670753e+07,8.0,1843,22.0,25343.204066,771.0,6,16.083333
839,"(CLT, ILM)",4.793792e+07,6.0,732,291.0,65488.962106,16.0,2,16.416667
2004,"(JFK, SFO)",4.663502e+07,9.0,1842,23.0,25317.600233,774.0,4,16.450000
